## Métodos de reamostragem

### Créditos

1. http://cursos.leg.ufpr.br/ML4all/apoio/reamostragem.html

2. https://dcm.ffclrp.usp.br/~augusto/teaching/ami/AM-I-Metodos-Amostragem.pdf


### Conteúdo

 - Holdout
 - Cross-Validation (also known as k-Fold Cross Validation)
  - Stratified Cross-Validation
  - Leave-One-Out
 - Bootstrap

Métodos de reamostragem são ferramentas indispensáveis na estatística moderna. As técnicas envolvem particionar os dados de treino e reajustar os modelos em competição para cada subamostra, a fim de obter informações adicionais sobre o ajuste do modelo (que não seria possível com os dados completos).

**Carregar dados**

In [1]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = data.data
y = data.target

**Validação Holdout**

A atividade envolve dividir aleatoriamente o conjunto de dados em (i) treinamento: utilizado para preparar o modelo (treiná-lo) e (ii) validação: utilizado para avaliar o desempenho do modelo treinado.

<img src="https://algotrading101.com/learn/wp-content/uploads/2020/06/training-validation-test-data-set.png" alt="drawing" width="600"/>

Source: https://algotrading101.com/learn/train-test-split/

In [2]:
from sklearn.model_selection import train_test_split

train, test, train_labels, test_labels = train_test_split(X,
                                                          y,
                                                          test_size=0.3,
                                                          random_state=12,
                                                          stratify=y)

In [10]:
test.shape
train.shape

(398, 30)

In [8]:
X.shape

(569, 30)

**Cross-Validation**

Separar os dados em somente duas partes disjuntas pode trazer resultados divergentes, dependendo da informação contida em cada conjunto (especialmente quando os dados são escassos). A abordagem de validação cruzada por k-fold minimiza esses problemas. O método consiste em dividir os dados em K partes iguais, ajusta-se o modelo utilizando K−1 partes, e a parcela restante fica destinada à validação. Esse processo é repetido K vezes (em cada momento uma partição diferente será a validação), em seguida os resultados são combinados obtendo a médias dos erros obtidos.

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" alt="drawing" width="600"/>

Source: https://scikit-learn.org/stable/modules/cross_validation.html

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

data = load_breast_cancer()
X = data.data
y = data.target

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())])

cv_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv = 10)

print("Cross-val accuracy: %f" % cv_scores.mean())

Cross-val accuracy: 0.964912


**Stratified Cross-Validation**

O estimador stratified cross-validation é similar à cross-validation, mas ao gerar os folds mutuamente exclusivos, a distribuição de classe (proporção de exemplos em cada uma das classes) é considerada durante a amostragem. Isto significa, por exemplo, que se o conjunto original de exemplos possui duas classes com distribuição de 20% e 80%, então cada fold também terá esta proporção de classes


<img src="https://dataaspirant.com/wp-content/uploads/2020/12/8-Stratified-K-Fold-Cross-Validation.png" alt="drawing" width="600"/>

Source: https://dataaspirant.com/8-stratified-k-fold-cross-validation/



In [15]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=20)
score = []
for train_index, test_index in folds.split(X, y):
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = y[train_index], y[test_index]
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    score.append(accuracy_score(y_val, model.predict(X_val)))
np.mean(score) 

0.9613408521303258

**Leave-One-Out**

O estimador leave-one-out é um caso especial de cross-validation. É computacionalmente dispendioso e freqüentemente é usado em amostras pequenas. Para uma amostra de tamanho n uma hipótese é induzida utilizando (n-1) exemplos; a hipótese é então testada no único exemplo remanescente. Este processo é repetido n vezes, cada vez induzindo uma hipótese deixando de considerar um único exemplo.

<img src="https://assets.datacamp.com/production/repositories/3981/datasets/8a6236f142b1ee2e4a70aae2af9507c7c580f302/Screen%20Shot%202019-01-27%20at%209.25.41%20AM.png" alt="drawing" width="600"/>

Source: https://campus.datacamp.com/courses/model-validation-in-python/cross-validation?ex=10

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
score = []
for train_index, test_index in loo.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    score.append(accuracy_score(y_test, model.predict(X_test)))
print(np.mean(score))

0.9560632688927944


In [ ]:
len(score)

569

**Bootstrap**

No estimador bootstrap, a idéia básica consiste em repetir o processo de classificação um grande número de vezes. Estima-se então valores, tais como o erro ou bias, a partir dos experimentos replicados, cada experimento sendo conduzido com base em um novo conjunto de treinamento obtido por amostragem com reposição do conjunto original de exemplos

![](http://rasbt.github.io/mlxtend/user_guide/evaluate/BootstrapOutOfBag_files/bootstrap_concept.png)

Source: http://rasbt.github.io/mlxtend/user_guide/evaluate/bootstrap_point632_score/

In [16]:
from sklearn.model_selection import ShuffleSplit

score = []
ss = ShuffleSplit(n_splits=1000, test_size=0.25, random_state=3)
for train_index, test_index in ss.split(X, y):
    # print("%s %s" % (train_index, test_index))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    score.append(accuracy_score(y_test, model.predict(X_test)))
print(np.mean(score))

0.9586923076923076
